In [ ]:
# importing the required libraries
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from matplotlib import pyplot as plt
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
from sklearn.feature_selection import RFE
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
#!pip install lime
import lime
import lime.lime_tabular

In [ ]:
df = pd.read_csv('data_regression.csv')
df

,year,customer_id,phone_no,gender,age,no_of_days_subscribed,multi_screen,mail_subscribed,weekly_mins_watched,minimum_daily_mins,maximum_daily_mins,weekly_max_night_mins,videos_watched,maximum_days_inactive,customer_support_calls,churn
0,2015,100198,409-8743,Female,36,62,no,no,148.35,12.2,16.81,82,1,4.0,1,0.0
1,2015,100643,340-5930,Female,39,149,no,no,294.45,7.7,33.37,87,3,3.0,2,0.0
2,2015,100756,372-3750,Female,65,126,no,no,87.30,11.9,9.89,91,1,4.0,5,1.0
3,2015,101595,331-4902,Female,24,131,no,yes,321.30,9.5,36.41,102,4,3.0,3,0.0
4,2015,101653,351-8398,Female,40,191,no,no,243.00,10.9,27.54,83,7,3.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,2015,997132,385-7387,Female,54,75,no,yes,182.25,11.3,20.66,97,5,4.0,2,NaN
1996,2015,998086,383-9255,Male,45,127,no,no,273.45,9.3,30.99,116,3,3.0,1,0.0
1997,2015,998474,353-2080,NaN,53,94,no,no,128.85,15.6,14.60,110,16,5.0,0,0.0
1998,2015,998934,359-7788,Male,40,94,no,no,178.05,10.4,20.18,100,6,NaN,3,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   year                    2000 non-null   int64  
 1   customer_id             2000 non-null   int64  
 2   phone_no                2000 non-null   object 
 3   gender                  1976 non-null   object 
 4   age                     2000 non-null   int64  
 5   no_of_days_subscribed   2000 non-null   int64  
 6   multi_screen            2000 non-null   object 
 7   mail_subscribed         2000 non-null   object 
 8   weekly_mins_watched     2000 non-null   float64
 9   minimum_daily_mins      2000 non-null   float64
 10  maximum_daily_mins      2000 non-null   float64
 11  weekly_max_night_mins   2000 non-null   int64  
 12  videos_watched          2000 non-null   int64  
 13  maximum_days_inactive   1972 non-null   float64
 14  customer_support_calls  2000 non-null   

In [ ]:
df.isnull().sum()

year                       0
customer_id                0
phone_no                   0
gender                    24
age                        0
no_of_days_subscribed      0
multi_screen               0
mail_subscribed            0
weekly_mins_watched        0
minimum_daily_mins         0
maximum_daily_mins         0
weekly_max_night_mins      0
videos_watched             0
maximum_days_inactive     28
customer_support_calls     0
churn                     35
dtype: int64

In [ ]:
df=df.dropna()

In [ ]:
df.select_dtypes(include=np.number).columns.tolist()

['year',
 'customer_id',
 'age',
 'no_of_days_subscribed',
 'weekly_mins_watched',
 'minimum_daily_mins',
 'maximum_daily_mins',
 'weekly_max_night_mins',
 'videos_watched',
 'maximum_days_inactive',
 'customer_support_calls',
 'churn']

In [ ]:
cols = df.select_dtypes(include=np.number).columns.tolist()

In [ ]:
df['churn'].value_counts()

churn
0.0    1665
1.0     253
Name: count, dtype: int64

In [ ]:
X=df[cols]
X

,year,customer_id,age,no_of_days_subscribed,weekly_mins_watched,minimum_daily_mins,maximum_daily_mins,weekly_max_night_mins,videos_watched,maximum_days_inactive,customer_support_calls,churn
0,2015,100198,36,62,148.35,12.2,16.81,82,1,4.0,1,0.0
1,2015,100643,39,149,294.45,7.7,33.37,87,3,3.0,2,0.0
2,2015,100756,65,126,87.30,11.9,9.89,91,1,4.0,5,1.0
3,2015,101595,24,131,321.30,9.5,36.41,102,4,3.0,3,0.0
4,2015,101653,40,191,243.00,10.9,27.54,83,7,3.0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1990,2015,993714,32,61,67.50,9.8,7.65,94,6,3.0,2,0.0
1991,2015,993815,49,50,460.65,8.0,52.21,109,3,3.0,0,0.0
1992,2015,994954,42,119,176.70,7.6,20.03,76,3,3.0,3,0.0
1996,2015,998086,45,127,273.45,9.3,30.99,116,3,3.0,1,0.0


In [ ]:
X.columns.difference(['churn'])

Index(['age', 'customer_id', 'customer_support_calls', 'maximum_daily_mins',
       'maximum_days_inactive', 'minimum_daily_mins', 'no_of_days_subscribed',
       'videos_watched', 'weekly_max_night_mins', 'weekly_mins_watched',
       'year'],
      dtype='object')

In [ ]:
X=X.drop(columns=['churn','customer_id','year'])

In [ ]:
X

,age,no_of_days_subscribed,weekly_mins_watched,minimum_daily_mins,maximum_daily_mins,weekly_max_night_mins,videos_watched,maximum_days_inactive,customer_support_calls
0,36,62,148.35,12.2,16.81,82,1,4.0,1
1,39,149,294.45,7.7,33.37,87,3,3.0,2
2,65,126,87.30,11.9,9.89,91,1,4.0,5
3,24,131,321.30,9.5,36.41,102,4,3.0,3
4,40,191,243.00,10.9,27.54,83,7,3.0,1
...,...,...,...,...,...,...,...,...,...
1990,32,61,67.50,9.8,7.65,94,6,3.0,2
1991,49,50,460.65,8.0,52.21,109,3,3.0,0
1992,42,119,176.70,7.6,20.03,76,3,3.0,3
1996,45,127,273.45,9.3,30.99,116,3,3.0,1


In [ ]:
y=df.churn

In [ ]:
y

0       0.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
1990    0.0
1991    0.0
1992    0.0
1996    0.0
1999    1.0
Name: churn, Length: 1918, dtype: float64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
X_train

,age,no_of_days_subscribed,weekly_mins_watched,minimum_daily_mins,maximum_daily_mins,weekly_max_night_mins,videos_watched,maximum_days_inactive,customer_support_calls
533,32,36,303.60,10.2,34.41,127,2,3.0,3
796,25,82,300.45,7.1,34.05,60,1,2.0,4
1180,46,87,250.95,11.0,28.44,88,6,3.0,1
506,20,193,145.20,10.9,16.46,115,5,3.0,2
77,43,140,187.95,8.4,21.30,94,4,3.0,1
...,...,...,...,...,...,...,...,...,...
897,42,94,245.25,12.6,27.80,82,5,4.0,1
1281,37,111,337.35,10.3,38.23,97,3,3.0,0
1723,38,103,161.55,8.9,18.31,98,3,3.0,0
613,37,25,201.45,12.6,22.83,100,5,4.0,2


In [ ]:
y_train

533     0.0
796     0.0
1180    0.0
506     1.0
77      0.0
       ... 
897     0.0
1281    0.0
1723    0.0
613     0.0
741     0.0
Name: churn, Length: 1342, dtype: float64

In [ ]:
X_test

,age,no_of_days_subscribed,weekly_mins_watched,minimum_daily_mins,maximum_daily_mins,weekly_max_night_mins,videos_watched,maximum_days_inactive,customer_support_calls
407,36,127,399.90,5.9,45.32,119,2,2.0,1
1620,37,107,171.45,7.5,19.43,74,8,3.0,1
1311,22,140,180.45,7.7,20.45,74,3,3.0,4
1875,30,66,131.40,9.2,14.89,125,5,3.0,1
476,30,134,214.35,10.8,24.29,96,6,3.0,1
...,...,...,...,...,...,...,...,...,...
1027,30,95,248.25,11.5,28.14,89,2,4.0,1
567,44,133,382.05,8.0,43.30,103,3,3.0,0
80,27,132,289.95,11.6,32.86,119,4,4.0,5
1999,37,73,326.70,10.3,37.03,89,6,3.0,1


In [ ]:
y_test

407     1.0
1620    0.0
1311    1.0
1875    0.0
476     0.0
       ... 
1027    1.0
567     1.0
80      1.0
1999    1.0
1717    0.0
Name: churn, Length: 576, dtype: float64

In [ ]:
sm = SMOTE(random_state=0, sampling_strategy=1.0)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [ ]:
X_train

,age,no_of_days_subscribed,weekly_mins_watched,minimum_daily_mins,maximum_daily_mins,weekly_max_night_mins,videos_watched,maximum_days_inactive,customer_support_calls
0,32,36,303.600000,10.200000,34.410000,127,2,3.000000,3
1,25,82,300.450000,7.100000,34.050000,60,1,2.000000,4
2,46,87,250.950000,11.000000,28.440000,88,6,3.000000,1
3,20,193,145.200000,10.900000,16.460000,115,5,3.000000,2
4,43,140,187.950000,8.400000,21.300000,94,4,3.000000,1
...,...,...,...,...,...,...,...,...,...
2299,52,178,245.999762,14.266144,27.881050,112,3,4.000000,1
2300,38,50,384.778836,12.833787,43.610583,107,5,3.842719,2
2301,34,132,260.796074,13.990484,29.556964,87,6,4.000000,1
2302,41,103,142.111273,8.700586,16.104301,93,4,2.607247,6


In [ ]:
y_train

0       0.0
1       0.0
2       0.0
3       1.0
4       0.0
       ... 
2299    1.0
2300    1.0
2301    1.0
2302    1.0
2303    1.0
Name: churn, Length: 2304, dtype: float64

## **Understanding SMOTE (Synthetic Minority Oversampling Technique)**

### **1️⃣ What is SMOTE?**
SMOTE (**Synthetic Minority Oversampling Technique**) is a method used to handle **imbalanced datasets** by creating **synthetic (artificial) data points** for the minority class.

- It **does not duplicate** existing data; instead, it **generates new synthetic data points** by interpolating between real samples.
- This technique is commonly used in machine learning problems where **one class (e.g., fraud detection, customer churn, rare disease detection) has far fewer samples than the other class**.

### **2️⃣ Why was SMOTE Introduced?**
In many real-world datasets, the number of instances in one class (e.g., "Churned Customers") is much lower than in another class (e.g., "Non-Churned Customers"). This causes **classification models to become biased toward the majority class**, making it difficult to correctly predict the minority class.

### **3️⃣ Why Can't We Just Use the Data As-Is?**
When training a machine learning model with an imbalanced dataset:
- The model may **ignore the minority class** because it appears too infrequently.
- Metrics like **accuracy** can be misleading because the model can predict the majority class most of the time and still get a high accuracy.

Example:

| Churn (Target) | Count |
|---------------|-------|
| No (0)        | 950   |
| Yes (1)       | 50    |

- If a model **always predicts "No"**, it will be **95% accurate**, but it **completely fails to detect churned customers**.
- **SMOTE helps by balancing the dataset** so the model can learn to predict the minority class better.

---

## **How SMOTE Works**
### **4️⃣ SMOTE vs. Traditional Oversampling**
- **Traditional Oversampling** duplicates minority class examples → ❌ **Overfitting Risk**.
- **SMOTE generates synthetic data** instead of duplicating → ✅ **Better Generalization**.

### **5️⃣ Steps in SMOTE**
1. **Finds the k-nearest neighbors** of a minority class sample.
2. **Randomly selects one of these neighbors**.
3. **Creates a synthetic data point** between the sample and the selected neighbor.
4. **Repeats the process** until the minority class is balanced.

### **Example of SMOTE**
#### **Before SMOTE: (Imbalanced Data)**
```
Class 0 (Majority) → ● ● ● ● ● ● ● ● ● ●
Class 1 (Minority) → ○ ○
```

#### **After SMOTE: (Balanced Data)**
```
Class 0 (Majority) → ● ● ● ● ● ● ● ● ● ●
Class 1 (SMOTE Added) → ○ ○ ○ ○ ○ ○ ○ ○ ○ ○
```
- **Synthetic (○) samples are added**, making the dataset more balanced.

---

## **6️⃣ Why Use SMOTE?**
| Problem | How SMOTE Helps |
|---------|---------------|
| **Model ignores the minority class** | Balances the dataset so the model learns both classes equally |
| **Overfitting with duplication-based oversampling** | Generates new synthetic data instead of duplicating existing data |
| **Biased predictions towards the majority class** | Improves generalization, preventing false assumptions |
| **Poor recall for minority class** | Helps detect the rare class better |

---

## **8️⃣ Where is SMOTE Used?**
SMOTE is commonly used in:
1. **Fraud detection** (credit card fraud, insurance fraud)
2. **Churn prediction** (identifying customers likely to leave)
3. **Disease detection** (predicting rare medical conditions)
4. **Spam filtering** (handling cases where spam emails are much less frequent than non-spam)
5. **Fault detection in manufacturing** (rare cases of machinery failure)

---

## **9️⃣ When Should You NOT Use SMOTE?**
While SMOTE is helpful, it should **not** be used in some cases:

❌ **If the minority class has very few examples (e.g., less than 5%)**
   - SMOTE can create **synthetic noise**, leading to poor model performance.

❌ **If features are categorical**
   - SMOTE works best with **numerical data** because it interpolates between values.
   - If your dataset has categorical features, use **SMOTE-NC (Nominal Continuous SMOTE)**.

❌ **If the dataset is already balanced**
   - If both classes have similar sizes, SMOTE is unnecessary.

---

## **🔹 Summary: SMOTE in a Nutshell**
| Feature | What It Does |
|---------|-------------|
| **Goal** | Balances imbalanced datasets by generating synthetic minority class examples |
| **Why?** | Prevents models from ignoring the minority class |
| **How?** | Creates new synthetic points by interpolating between existing minority class samples |
| **When to Use?** | When class imbalance is significant (e.g., 90-10 ratio) |
| **When NOT to Use?** | When dataset is small, categorical, or already balanced |

Would you like a **detailed visualization of how SMOTE works**? 🚀

In [ ]:
log_reg_model = LogisticRegression(random_state = 13)
log_reg_model.fit(X_train, y_train)

LogisticRegression(random_state=13)

In [ ]:
y_pred = log_reg_model.predict(X_test)

In [ ]:
logit_roc_auc = roc_auc_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import roc_auc_score,classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.94      0.64      0.76       513
         1.0       0.19      0.67      0.29        63

    accuracy                           0.64       576
   macro avg       0.56      0.65      0.53       576
weighted avg       0.86      0.64      0.71       576



### Outcome and Analysis of the Classification Report:

#### Class 0 (No Churn)

| **Metric**   | **Value** | **Meaning** |
|-------------|---------|--------------------------------------------------------------|
| **Precision** | 0.94    | When the model predicts "No Churn," it is correct 94% of the time. |
| **Recall**    | 0.64    | Out of all 513 actual "No Churn" customers, it correctly predicted 64%. |
| **F1-Score**  | 0.76    | A balance between precision and recall. |
| **Support**   | 513     | The number of "No Churn" samples in the test data. |

📌 **Issue:** Even though precision is high (94%), the recall is only 64%, meaning the model is missing 36% of "No Churn" customers.


#### Class 1 (Churn)

| **Metric**   | **Value** | **Meaning** |
|-------------|---------|--------------------------------------------------------------|
| **Precision** | 0.19    | When the model predicts "Churn," it is correct only 19% of the time! |
| **Recall**    | 0.67    | Out of 63 actual churners, it correctly found 67% of them. |
| **F1-Score**  | 0.29    | The balance of precision and recall is low. |
| **Support**   | 63      | The number of actual churn cases in the test data. |

📌 **Problem:** The model finds 67% of churn cases (good recall) but makes too many false positives, meaning most of its churn predictions are wrong (**precision = 19%**).


#### 📊 Overall Model Performance

| **Metric**    | **Value** | **Meaning (Simple Explanation)** |
|--------------|---------|--------------------------------------------------------------|
| **Accuracy**  | 0.64    | The model is correct 64% of the time (but can be misleading in imbalanced datasets). |
| **Macro Avg** | **Precision:** 0.56  <br> **Recall:** 0.65  <br> **F1-Score:** 0.53 | The average of Precision, Recall, and F1-Score across both classes (treats both equally). |
| **Weighted Avg** | **Precision:** 0.86  <br> **Recall:** 0.64  <br> **F1-Score:** 0.71 | Accounts for class imbalance by giving more weight to the majority class (Class 0 has more samples). |

📌 Issue: The accuracy (64%) is misleading because the model is very good at detecting No Churn (0) but bad at detecting Churn (1).

#### 🔍 Key Observations  
- **High Precision for Class 0 (0.94)** → When the model predicts "No Churn," it's right most of the time.  
- **Low Precision for Class 1 (0.19)** → When the model predicts "Churn," it's wrong 81% of the time.  
- **Good Recall for Class 1 (0.67)** → The model correctly identifies 67% of actual churn cases.  
- **Accuracy (0.64) is misleading** → Because of class imbalance, accuracy alone is not a good metric.  
- **F1-score for Churn (0.29) is poor** → The model struggles with predicting the minority class.  



In [ ]:
logit_roc_auc

np.float64(0.6539961013645224)

## Decision Tree Model

In [ ]:
dectree = DecisionTreeClassifier(random_state = 13,criterion = 'entropy')
dectree.fit(X_train, y_train)

##Predicting y values
y_pred = dectree.predict(X_test)
dectree_roc_auc = roc_auc_score(y_test, dectree.predict(X_test))
print(classification_report(y_test, y_pred))
print("The area under the curve is: %0.2f"%dectree_roc_auc)
print(f'\nprobabilities for the target (0 and 1) is \n{randomforest.predict_proba(X_test)}')

              precision    recall  f1-score   support

         0.0       0.94      0.85      0.89       513
         1.0       0.31      0.57      0.40        63

    accuracy                           0.82       576
   macro avg       0.63      0.71      0.65       576
weighted avg       0.87      0.82      0.84       576

The area under the curve is: 0.71

probabilities for the target (0 and 1) is 
[[0.29100302 0.70899698]
 [0.81881769 0.18118231]
 [0.21644917 0.78355083]
 ...
 [0.62070984 0.37929016]
 [0.78476201 0.21523799]
 [0.65200993 0.34799007]]


### **📊 Classification Report Breakdown for Decision Tree**
The model used here is a **Decision Tree Classifier** with the `entropy` criterion.

---

### **1️⃣ Class-wise Performance Metrics**
| **Metric**        | **Class 0 (No Churn)** | **Class 1 (Churn)** | **Meaning (Simple Explanation)** |
|-------------------|----------------------|---------------------|--------------------------------|
| **Precision**    | **0.94**              | **0.31**            | Out of all predictions for this class, how many were correct? (Higher is better) |
| **Recall**       | **0.85**              | **0.57**            | Out of all actual instances of this class, how many were correctly predicted? (Higher is better) |
| **F1-Score**     | **0.89**              | **0.40**            | The harmonic mean of Precision & Recall (Balances both metrics) |
| **Support**      | **513**               | **63**              | The actual number of samples in `y_test` for each class |

---

### **2️⃣ Overall Model Performance**
| **Metric**         | **Value**  | **Meaning (Simple Explanation)** |
|-------------------|-----------|--------------------------------|
| **Accuracy**      | **0.82**   | The model is correct **82% of the time** (but can be misleading in imbalanced datasets). |
| **Macro Avg**     | Precision: **0.63**<br>Recall: **0.71**<br>F1-Score: **0.65** | The average of Precision, Recall, and F1-Score across both classes (treats both equally). |
| **Weighted Avg**  | Precision: **0.87**<br>Recall: **0.82**<br>F1-Score: **0.84** | Accounts for class imbalance by giving more weight to the majority class (Class 0 has more samples). |
| **ROC-AUC Score** | **0.71**   | Measures the model’s ability to distinguish between classes. (Higher is better; 0.5 = random chance). |

---

## **🔄 Comparison: Logistic Regression vs. Decision Tree**
### **📊 Comparing Class-wise Metrics**
| **Metric**        | **Logistic Regression (Previous Model)** | **Decision Tree (Current Model)** | **Improvement?** |
|-------------------|----------------------------------|----------------------------|--------------|
| **Precision (Class 0)** | **0.94** | **0.94** | 🔹 No Change |
| **Precision (Class 1)** | **0.19** | **0.31** | ✅ Improved (better at correctly identifying churn cases) |
| **Recall (Class 0)** | **0.64** | **0.85** | ✅ Improved (fewer false negatives for no churn) |
| **Recall (Class 1)** | **0.67** | **0.57** | ❌ Decreased (misses more actual churn cases) |
| **F1-Score (Class 0)** | **0.76** | **0.89** | ✅ Improved |
| **F1-Score (Class 1)** | **0.29** | **0.40** | ✅ Improved (but still low) |
| **Support (Class 0)** | 513 | 513 | 🔹 No Change |
| **Support (Class 1)** | 63 | 63 | 🔹 No Change |

---

### **📊 Comparing Overall Metrics**
| **Metric**        | **Logistic Regression** | **Decision Tree** | **Improvement?** |
|-------------------|------------------|------------------|--------------|
| **Accuracy**      | **0.64** | **0.82** | ✅ Improved |
| **Macro Avg Precision** | **0.56** | **0.63** | ✅ Improved |
| **Macro Avg Recall** | **0.65** | **0.71** | ✅ Improved |
| **Macro Avg F1-Score** | **0.53** | **0.65** | ✅ Improved |
| **Weighted Avg Precision** | **0.86** | **0.87** | 🔹 Slight Improvement |
| **Weighted Avg Recall** | **0.64** | **0.82** | ✅ Improved |
| **Weighted Avg F1-Score** | **0.71** | **0.84** | ✅ Improved |
| **ROC-AUC Score** | **0.71** | **0.71** | 🔹 No Change |

---

## **📌 Discussion: Did Decision Tree Improve Performance?**
### ✅ **Improvements with Decision Tree:**
1. **Overall Accuracy increased from 64% → 82%**  
   - **Better than logistic regression** in overall correctness.
   
2. **Higher Recall for Class 0 (No Churn) → 85% (vs. 64%)**  
   - The Decision Tree is better at **correctly classifying No Churn customers**.

3. **Precision for Class 1 (Churn) increased from 19% → 31%**  
   - The Decision Tree makes **fewer false churn predictions** than logistic regression.

4. **F1-score for Class 1 (Churn) improved from 0.29 → 0.40**  
   - This means the model is getting better at detecting churners.

---

### ❌ **Problems with Decision Tree:**
1. **Recall for Churn (Class 1) dropped from 67% → 57%**  
   - It **misses more actual churners** than logistic regression.
   - This could be because decision trees **overfit the majority class (No Churn)**.

2. **ROC-AUC Score stayed the same at 0.71**  
   - While the accuracy is better, the **model is not significantly better at ranking predictions**.



## **🔍 Final Conclusion: Should We Use the Decision Tree Instead of Logistic Regression?**
| **Aspect** | **Logistic Regression** | **Decision Tree** | **Better Model?** |
|------------|------------------------|-------------------|----------------|
| **Overall Accuracy** | **64%** | **82%** | ✅ Decision Tree |
| **Detecting Churn (Class 1) - Precision** | **19%** | **31%** | ✅ Decision Tree |
| **Detecting Churn (Class 1) - Recall** | **67%** | **57%** | ❌ Logistic Regression |
| **F1-score for Churn (Class 1)** | **0.29** | **0.40** | ✅ Decision Tree |
| **Overfitting Risk** | **Low (Generalizes Well)** | **High (May Overfit)** | ❌ Logistic Regression |
| **ROC-AUC Score** | **0.71** | **0.71** | 🔹 No Change |

### **📌 Which Model Should You Use?**
- If you **only care about overall accuracy**, **Decision Tree is better**.
- If you **care more about recall for detecting churn (reducing false negatives)**, **Logistic Regression is better**.
- Decision Trees **tend to overfit** on imbalanced data, so we may need **pruning or ensemble methods (Random Forest, XGBoost)**.

---

### **🚀 Next Steps for Model Improvement**
1. **Try a Random Forest or XGBoost model** (to reduce overfitting in Decision Trees).
2. **Use SMOTE** to balance the dataset before training.
3. **Hyperparameter tuning** (limit max depth of Decision Tree to prevent overfitting).
4. **Adjust the threshold** (lower threshold for predicting churn in Logistic Regression).

Would you like me to implement **Random Forest** and compare the results? 🚀

## Random Forest

In [ ]:
randomforest = RandomForestClassifier(max_depth=5)
randomforest.fit(X_train, y_train)
##Predicting y values
y_pred = randomforest.predict(X_test)
randomforest_roc_auc = roc_auc_score(y_test, randomforest.predict(X_test))
print(classification_report(y_test, y_pred))
print("The area under the curve is: %0.2f"%randomforest_roc_auc)

              precision    recall  f1-score   support

         0.0       0.95      0.88      0.91       513
         1.0       0.39      0.60      0.47        63

    accuracy                           0.85       576
   macro avg       0.67      0.74      0.69       576
weighted avg       0.89      0.85      0.87       576

The area under the curve is: 0.74


## **📊 Classification Report Breakdown for Random Forest**
The model used here is a **Random Forest Classifier** with `max_depth=5`, meaning it limits the depth of individual decision trees to prevent overfitting.

---

### **1️⃣ Class-wise Performance Metrics**
| **Metric**        | **Class 0 (No Churn)** | **Class 1 (Churn)** | **Meaning (Simple Explanation)** |
|-------------------|----------------------|---------------------|--------------------------------|
| **Precision**    | **0.95**              | **0.39**            | Out of all predictions for this class, how many were correct? (Higher is better) |
| **Recall**       | **0.88**              | **0.60**            | Out of all actual instances of this class, how many were correctly predicted? (Higher is better) |
| **F1-Score**     | **0.91**              | **0.47**            | The harmonic mean of Precision & Recall (Balances both metrics) |
| **Support**      | **513**               | **63**              | The actual number of samples in `y_test` for each class |

---

### **2️⃣ Overall Model Performance**
| **Metric**         | **Value**  | **Meaning (Simple Explanation)** |
|-------------------|-----------|--------------------------------|
| **Accuracy**      | **0.85**   | The model is correct **85% of the time**. |
| **Macro Avg**     | Precision: **0.67**<br>Recall: **0.74**<br>F1-Score: **0.69** | The average of Precision, Recall, and F1-Score across both classes (treats both equally). |
| **Weighted Avg**  | Precision: **0.89**<br>Recall: **0.85**<br>F1-Score: **0.87** | Accounts for class imbalance by giving more weight to the majority class (Class 0 has more samples). |
| **ROC-AUC Score** | **0.74**   | Measures the model’s ability to distinguish between classes. (Higher is better; 0.5 = random chance). |

---

## **🔄 Comparison: Logistic Regression vs. Decision Tree vs. Random Forest**
### **📊 Comparing Class-wise Metrics**
| **Metric**        | **Logistic Regression** | **Decision Tree** | **Random Forest** | **Best Model?** |
|-------------------|------------------|----------------|----------------|--------------|
| **Precision (Class 0)** | **0.94** | **0.94** | **0.95** | ✅ Random Forest |
| **Precision (Class 1)** | **0.19** | **0.31** | **0.39** | ✅ Random Forest |
| **Recall (Class 0)** | **0.64** | **0.85** | **0.88** | ✅ Random Forest |
| **Recall (Class 1)** | **0.67** | **0.57** | **0.60** | ✅ Logistic Regression (slightly better) |
| **F1-Score (Class 0)** | **0.76** | **0.89** | **0.91** | ✅ Random Forest |
| **F1-Score (Class 1)** | **0.29** | **0.40** | **0.47** | ✅ Random Forest |
| **Support (Class 0)** | 513 | 513 | 513 | 🔹 No Change |
| **Support (Class 1)** | 63 | 63 | 63 | 🔹 No Change |

---

### **📊 Comparing Overall Metrics**
| **Metric**        | **Logistic Regression** | **Decision Tree** | **Random Forest** | **Best Model?** |
|-------------------|------------------|----------------|----------------|--------------|
| **Accuracy**      | **0.64** | **0.82** | **0.85** | ✅ Random Forest |
| **Macro Avg Precision** | **0.56** | **0.63** | **0.67** | ✅ Random Forest |
| **Macro Avg Recall** | **0.65** | **0.71** | **0.74** | ✅ Random Forest |
| **Macro Avg F1-Score** | **0.53** | **0.65** | **0.69** | ✅ Random Forest |
| **Weighted Avg Precision** | **0.86** | **0.87** | **0.89** | ✅ Random Forest |
| **Weighted Avg Recall** | **0.64** | **0.82** | **0.85** | ✅ Random Forest |
| **Weighted Avg F1-Score** | **0.71** | **0.84** | **0.87** | ✅ Random Forest |
| **ROC-AUC Score** | **0.71** | **0.71** | **0.74** | ✅ Random Forest |

---

## **📌 Discussion: Did Random Forest Improve Performance?**
### ✅ **Improvements with Random Forest:**
1. **Overall Accuracy increased from 64% (Logistic) → 82% (Decision Tree) → 85% (Random Forest)**  
   - **Random Forest performs best in overall correctness.**

2. **Precision for Churn (Class 1) increased from 19% (Logistic) → 31% (Decision Tree) → 39% (Random Forest)**  
   - This means **fewer false churn predictions** compared to previous models.

3. **Recall for Churn (Class 1) improved compared to Decision Tree**  
   - **57% (Decision Tree) → 60% (Random Forest)**
   - **(Still lower than Logistic Regression, which was 67%)**

4. **ROC-AUC Score improved from 0.71 → 0.74**  
   - Meaning the **model is slightly better at ranking churn vs. non-churn customers**.

---

### ❌ **Remaining Problems with Random Forest:**
1. **Still Low Precision for Churn (Class 1)**
   - Precision **0.39** means **many false positives** (incorrect churn predictions).
   - We need to fine-tune the model (e.g., increase `max_depth`, use feature selection).

2. **Still Lower Recall for Churn than Logistic Regression**
   - **67% (Logistic) → 60% (Random Forest)**
   - The model still **misses some actual churners**, meaning recall can be improved.

---

## **📌 Final Conclusion: Which Model is Best?**
| **Scenario** | **Best Model** | **Why?** |
|-------------|---------------|----------|
| **Overall Accuracy** | ✅ **Random Forest** (85%) | Best overall performance. |
| **Detecting Churn (Recall for Class 1)** | ✅ **Logistic Regression (67%)** | Finds more actual churn cases. |
| **Predicting Churn Correctly (Precision for Class 1)** | ✅ **Random Forest (39%)** | Fewer false positives. |
| **Generalization (Avoid Overfitting)** | ✅ **Random Forest** | Better regularization. |
| **ROC-AUC Score** | ✅ **Random Forest (0.74)** | Best ranking capability. |

---

## **🚀 Next Steps to Improve Performance**
1. **Fine-tune Random Forest**  
   - Increase `max_depth`, adjust `n_estimators` to find the best settings.

2. **Use SMOTE or Class Weighting**  

3. **Use XGBoost (Better for Imbalanced Data)**

## Ada Boost

In [ ]:
adaboost = AdaBoostClassifier(n_estimators = 100)
adaboost.fit(X_train, y_train)
##Predicting y values
global y_pred #Defines the Y_Pred as a global variable that can be used outside of this function
y_pred = adaboost.predict(X_test)
adaboost_roc_auc = roc_auc_score(y_test, adaboost.predict(X_test))
print(classification_report(y_test, y_pred))
print("The area under the curve is: %0.2f"%adaboost_roc_auc)

              precision    recall  f1-score   support

         0.0       0.95      0.88      0.92       513
         1.0       0.39      0.60      0.47        63

    accuracy                           0.85       576
   macro avg       0.67      0.74      0.70       576
weighted avg       0.89      0.85      0.87       576

The area under the curve is: 0.74


## **📊 Classification Report Breakdown for AdaBoost**
The model used here is **AdaBoostClassifier** with `n_estimators=100`, meaning it combines 100 weak classifiers (usually decision trees) to create a strong model.

---

### **1️⃣ Class-wise Performance Metrics**
| **Metric**        | **Class 0 (No Churn)** | **Class 1 (Churn)** | **Meaning (Simple Explanation)** |
|-------------------|----------------------|---------------------|--------------------------------|
| **Precision**    | **0.95**              | **0.39**            | Out of all predictions for this class, how many were correct? (Higher is better) |
| **Recall**       | **0.88**              | **0.60**            | Out of all actual instances of this class, how many were correctly predicted? (Higher is better) |
| **F1-Score**     | **0.92**              | **0.47**            | The harmonic mean of Precision & Recall (Balances both metrics) |
| **Support**      | **513**               | **63**              | The actual number of samples in `y_test` for each class |

---

### **2️⃣ Overall Model Performance**
| **Metric**         | **Value**  | **Meaning (Simple Explanation)** |
|-------------------|-----------|--------------------------------|
| **Accuracy**      | **0.85**   | The model is correct **85% of the time**. |
| **Macro Avg**     | Precision: **0.67**<br>Recall: **0.74**<br>F1-Score: **0.70** | The average of Precision, Recall, and F1-Score across both classes (treats both equally). |
| **Weighted Avg**  | Precision: **0.89**<br>Recall: **0.85**<br>F1-Score: **0.87** | Accounts for class imbalance by giving more weight to the majority class (Class 0 has more samples). |
| **ROC-AUC Score** | **0.74**   | Measures the model’s ability to distinguish between classes. (Higher is better; 0.5 = random chance). |

---

## **🔄 Comparison: Logistic Regression vs. Decision Tree vs. Random Forest vs. AdaBoost**
### **📊 Comparing Class-wise Metrics**
| **Metric**        | **Logistic Regression** | **Decision Tree** | **Random Forest** | **AdaBoost** | **Best Model?** |
|-------------------|------------------|----------------|----------------|----------------|--------------|
| **Precision (Class 0)** | **0.94** | **0.94** | **0.95** | **0.95** | 🔹 No Significant Change |
| **Precision (Class 1)** | **0.19** | **0.31** | **0.39** | **0.39** | ✅ Random Forest / AdaBoost |
| **Recall (Class 0)** | **0.64** | **0.85** | **0.88** | **0.88** | ✅ Random Forest / AdaBoost |
| **Recall (Class 1)** | **0.67** | **0.57** | **0.60** | **0.60** | ✅ Logistic Regression |
| **F1-Score (Class 0)** | **0.76** | **0.89** | **0.91** | **0.92** | ✅ AdaBoost |
| **F1-Score (Class 1)** | **0.29** | **0.40** | **0.47** | **0.47** | ✅ Random Forest / AdaBoost |
| **Support (Class 0)** | 513 | 513 | 513 | 513 | 🔹 No Change |
| **Support (Class 1)** | 63 | 63 | 63 | 63 | 🔹 No Change |

---

### **📊 Comparing Overall Metrics**
| **Metric**        | **Logistic Regression** | **Decision Tree** | **Random Forest** | **AdaBoost** | **Best Model?** |
|-------------------|------------------|----------------|----------------|----------------|--------------|
| **Accuracy**      | **0.64** | **0.82** | **0.85** | **0.85** | ✅ Random Forest / AdaBoost |
| **Macro Avg Precision** | **0.56** | **0.63** | **0.67** | **0.67** | ✅ Random Forest / AdaBoost |
| **Macro Avg Recall** | **0.65** | **0.71** | **0.74** | **0.74** | ✅ Random Forest / AdaBoost |
| **Macro Avg F1-Score** | **0.53** | **0.65** | **0.69** | **0.70** | ✅ AdaBoost |
| **Weighted Avg Precision** | **0.86** | **0.87** | **0.89** | **0.89** | ✅ Random Forest / AdaBoost |
| **Weighted Avg Recall** | **0.64** | **0.82** | **0.85** | **0.85** | ✅ Random Forest / AdaBoost |
| **Weighted Avg F1-Score** | **0.71** | **0.84** | **0.87** | **0.87** | ✅ Random Forest / AdaBoost |
| **ROC-AUC Score** | **0.71** | **0.71** | **0.74** | **0.74** | ✅ Random Forest / AdaBoost |

---

## **📌 Discussion: Did AdaBoost Improve Performance?**
### ✅ **Improvements with AdaBoost:**
1. **Overall Accuracy (85%) is the same as Random Forest, better than Decision Tree and Logistic Regression.**
2. **High Precision for Class 0 (0.95)** → Fewer false positives for "No Churn" cases.
3. **Better Recall for Class 1 (0.60) than Decision Tree (0.57)** → Slightly better at detecting churn cases than a single decision tree.
4. **F1-score for Churn (0.47) matches Random Forest** → Meaning AdaBoost is **as good as Random Forest** in balancing precision and recall.

---

### ❌ **Limitations of AdaBoost**
1. **Still Low Precision for Churn (Class 1)**
   - Precision **0.39**, meaning many false positives.
   - This can be improved by **adjusting the class weights** or tuning the model further.

2. **Recall for Churn (Class 1) is Still Lower Than Logistic Regression**
   - **67% (Logistic) → 60% (AdaBoost)**
   - Logistic Regression **still identifies more churn cases correctly**, even if it has more false positives.

3. **Not a Big Difference from Random Forest**
   - Since AdaBoost builds trees sequentially, it often performs similarly to **Random Forest**.
   - If **training time is an issue**, **Random Forest is preferred** since it's parallelizable.

---

## **📌 Final Conclusion: Which Model is Best?**
| **Scenario** | **Best Model** | **Why?** |
|-------------|---------------|----------|
| **Overall Accuracy** | ✅ **Random Forest / AdaBoost (85%)** | Best overall performance. |
| **Detecting Churn (Recall for Class 1)** | ✅ **Logistic Regression (67%)** | Finds more actual churn cases. |
| **Predicting Churn Correctly (Precision for Class 1)** | ✅ **Random Forest / AdaBoost (39%)** | Fewer false positives. |
| **Generalization (Avoid Overfitting)** | ✅ **Random Forest** | More stable performance. |
| **ROC-AUC Score** | ✅ **Random Forest / AdaBoost (0.74)** | Best ranking capability. |

---

## **🚀 Next Steps to Improve Performance**
1. **Fine-tune AdaBoost (`learning_rate`, `n_estimators`)**  
   - Higher `n_estimators` can improve performance, but may increase overfitting.
2. **Use SMOTE or Class Weighting**  

3. **Try XGBoost (More Powerful Boosting Algorithm)**

---

## **Final Recommendation**
- **Use AdaBoost if you want a stable, boosting-based model with better generalization.**
- **Use Random Forest for faster performance with similar accuracy.**
- **Use Logistic Regression if recall (finding churn cases) is your top priority.**


## Gradient Boost

In [ ]:
gradientboost = GradientBoostingClassifier()
gradientboost.fit(X_train, y_train)
#Predicting y values
global y_pred #Defines the Y_Pred as a global variable that can be used outside of this function
y_pred = gradientboost.predict(X_test)
gradientboost_roc_auc = roc_auc_score(y_test, gradientboost.predict(X_test))
print(classification_report(y_test, y_pred))
print("The area under the curve is: %0.2f"%gradientboost_roc_auc)

              precision    recall  f1-score   support

         0.0       0.95      0.90      0.93       513
         1.0       0.44      0.60      0.51        63

    accuracy                           0.87       576
   macro avg       0.69      0.75      0.72       576
weighted avg       0.89      0.87      0.88       576

The area under the curve is: 0.75


## **📊 Classification Report Breakdown for Gradient Boosting**
The model used here is **GradientBoostingClassifier()**, an ensemble learning method that builds trees sequentially to correct errors from previous ones.

---

### **1️⃣ Class-wise Performance Metrics**
| **Metric**        | **Class 0 (No Churn)** | **Class 1 (Churn)** | **Meaning (Simple Explanation)** |
|-------------------|----------------------|---------------------|--------------------------------|
| **Precision**    | **0.95**              | **0.44**            | Out of all predictions for this class, how many were correct? (Higher is better) |
| **Recall**       | **0.90**              | **0.60**            | Out of all actual instances of this class, how many were correctly predicted? (Higher is better) |
| **F1-Score**     | **0.93**              | **0.51**            | The harmonic mean of Precision & Recall (Balances both metrics) |
| **Support**      | **513**               | **63**              | The actual number of samples in `y_test` for each class |

---

### **2️⃣ Overall Model Performance**
| **Metric**         | **Value**  | **Meaning (Simple Explanation)** |
|-------------------|-----------|--------------------------------|
| **Accuracy**      | **0.87**   | The model is correct **87% of the time**. |
| **Macro Avg**     | Precision: **0.69**<br>Recall: **0.75**<br>F1-Score: **0.72** | The average of Precision, Recall, and F1-Score across both classes (treats both equally). |
| **Weighted Avg**  | Precision: **0.89**<br>Recall: **0.87**<br>F1-Score: **0.88** | Accounts for class imbalance by giving more weight to the majority class (Class 0 has more samples). |
| **ROC-AUC Score** | **0.75**   | Measures the model’s ability to distinguish between classes. (Higher is better; 0.5 = random chance). |



## **📌 Discussion: Did Gradient Boosting Improve Performance?**
### ✅ **Improvements with Gradient Boosting:**
1. **Overall Accuracy (87%) is the highest among all models.**
2. **Best Precision for Churn (0.44) compared to previous models (max was 0.39).**
3. **Recall for Churn (0.60) is same as Random Forest and AdaBoost, higher than Decision Tree (0.57).**
4. **Highest ROC-AUC Score (0.75) among all models.**
5. **Highest Macro Avg F1-Score (0.72), indicating the best balance between Precision and Recall.**

---

### ❌ **Limitations of Gradient Boosting**
1. **Still lower Recall for Churn (0.60) than Logistic Regression (0.67).**
   - Meaning it still misses some actual churners.
2. **Computationally expensive.**
   - Gradient Boosting takes **longer to train** compared to Logistic Regression or Decision Trees.

---

## **🚀 Next Steps to Improve Performance**
1. **Fine-tune Gradient Boosting (`n_estimators`, `learning_rate`)**  
   - Try increasing `n_estimators=200` and decreasing `learning_rate=0.05`.
2. **Use SMOTE or Class Weighting**  

3. **Try XGBoost (More Optimized Version of Gradient Boosting)**

---

## **Final Recommendation**
- **Gradient Boosting is the best overall model so far.**
- **XGBoost may improve performance further.**
- **Would you like to compare with XGBoost next?** 🚀

## XGBoost

In [ ]:
# Import necessary libraries
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Initialize and train the XGBoost classifier
xgboost = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgboost.fit(X_train, y_train)

# Predicting y values
y_pred = xgboost.predict(X_test)

# Calculate ROC-AUC Score
xgboost_roc_auc = roc_auc_score(y_test, xgboost.predict(X_test))

# Print Classification Report
print(classification_report(y_test, y_pred))
print("The area under the curve is: %0.2f" % xgboost_roc_auc)


              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93       513
         1.0       0.44      0.48      0.46        63

    accuracy                           0.88       576
   macro avg       0.69      0.70      0.69       576
weighted avg       0.88      0.88      0.88       576

The area under the curve is: 0.70


### **📊 Classification Report Breakdown for XGBoost**
The model used here is **XGBoostClassifier**, a powerful gradient boosting algorithm optimized for efficiency and performance.

---

#### **1️⃣ Class-wise Performance Metrics**
| **Metric**        | **Class 0 (No Churn)** | **Class 1 (Churn)** | **Meaning (Simple Explanation)** |
|-------------------|----------------------|---------------------|--------------------------------|
| **Precision**    | **0.94**              | **0.44**            | Out of all predictions for this class, how many were correct? (Higher is better) |
| **Recall**       | **0.93**              | **0.48**            | Out of all actual instances of this class, how many were correctly predicted? (Higher is better) |
| **F1-Score**     | **0.93**              | **0.46**            | The harmonic mean of Precision & Recall (Balances both metrics) |
| **Support**      | **513**               | **63**              | The actual number of samples in `y_test` for each class |

---

#### **2️⃣ Overall Model Performance**
| **Metric**         | **Value**  | **Meaning (Simple Explanation)** |
|-------------------|-----------|--------------------------------|
| **Accuracy**      | **0.88**   | The model is correct **88% of the time**. |
| **Macro Avg**     | Precision: **0.69**<br>Recall: **0.70**<br>F1-Score: **0.69** | The average of Precision, Recall, and F1-Score across both classes (treats both equally). |
| **Weighted Avg**  | Precision: **0.88**<br>Recall: **0.88**<br>F1-Score: **0.88** | Accounts for class imbalance by giving more weight to the majority class (Class 0 has more samples). |
| **ROC-AUC Score** | **0.70**   | Measures the model’s ability to distinguish between classes. (Higher is better; 0.5 = random chance). |

---

### **🔄 Comparison: Logistic Regression vs. Decision Tree vs. Random Forest vs. AdaBoost vs. Gradient Boosting vs. XGBoost**
### **📊 Comparing Class-wise Metrics**
| **Metric**        | **Logistic Regression** | **Decision Tree** | **Random Forest** | **AdaBoost** | **Gradient Boosting** | **XGBoost** | **Best Model?** |
|-------------------|------------------|----------------|----------------|----------------|----------------|----------------|--------------|
| **Precision (Class 0)** | **0.94** | **0.94** | **0.95** | **0.95** | **0.95** | **0.94** | 🔹 No Significant Change |
| **Precision (Class 1)** | **0.19** | **0.31** | **0.39** | **0.39** | **0.44** | **0.44** | ✅ Gradient Boosting / XGBoost |
| **Recall (Class 0)** | **0.64** | **0.85** | **0.88** | **0.88** | **0.90** | **0.93** | ✅ XGBoost |
| **Recall (Class 1)** | **0.67** | **0.57** | **0.60** | **0.60** | **0.60** | **0.48** | ✅ Logistic Regression (best recall) |
| **F1-Score (Class 0)** | **0.76** | **0.89** | **0.91** | **0.92** | **0.93** | **0.93** | ✅ Gradient Boosting / XGBoost |
| **F1-Score (Class 1)** | **0.29** | **0.40** | **0.47** | **0.47** | **0.51** | **0.46** | ✅ Gradient Boosting |

---

#### **📊 Comparing Overall Metrics**
| **Metric**        | **Logistic Regression** | **Decision Tree** | **Random Forest** | **AdaBoost** | **Gradient Boosting** | **XGBoost** | **Best Model?** |
|-------------------|------------------|----------------|----------------|----------------|----------------|----------------|--------------|
| **Accuracy**      | **0.64** | **0.82** | **0.85** | **0.85** | **0.87** | **0.88** | ✅ XGBoost |
| **Macro Avg Precision** | **0.56** | **0.63** | **0.67** | **0.67** | **0.69** | **0.69** | ✅ Gradient Boosting / XGBoost |
| **Macro Avg Recall** | **0.65** | **0.71** | **0.74** | **0.74** | **0.75** | **0.70** | ✅ Gradient Boosting |
| **Macro Avg F1-Score** | **0.53** | **0.65** | **0.69** | **0.70** | **0.72** | **0.69** | ✅ Gradient Boosting |
| **Weighted Avg Precision** | **0.86** | **0.87** | **0.89** | **0.89** | **0.89** | **0.88** | 🔹 No Change |
| **Weighted Avg Recall** | **0.64** | **0.82** | **0.85** | **0.85** | **0.87** | **0.88** | ✅ XGBoost |
| **Weighted Avg F1-Score** | **0.71** | **0.84** | **0.87** | **0.87** | **0.88** | **0.88** | 🔹 No Change |
| **ROC-AUC Score** | **0.71** | **0.71** | **0.74** | **0.74** | **0.75** | **0.70** | ✅ Gradient Boosting |

---

### **📌 Discussion: Did XGBoost Improve Performance?**
#### ✅ **Improvements with XGBoost:**
1. **Highest Overall Accuracy (88%)**
   - Slightly better than Gradient Boosting (87%).
   
2. **Best Recall for No Churn (93%)**
   - Meaning fewer false negatives for No Churn customers.

3. **Good Precision for Churn (0.44)**
   - Same as Gradient Boosting, better than other models.

4. **Good F1-score for Churn (0.46)**
   - Almost as good as Gradient Boosting.

---

#### ❌ **Limitations of XGBoost**
1. **Lower Recall for Churn (0.48) than Gradient Boosting (0.60)**
   - It **misses more actual churners** than Gradient Boosting.

2. **Slightly Lower ROC-AUC Score (0.70) Compared to Gradient Boosting (0.75)**
   - Meaning **Gradient Boosting ranks churn cases better**.

---

### **📌 Final Conclusion: Which Model is Best?**
| **Scenario** | **Best Model** | **Why?** |
|-------------|---------------|----------|
| **Best overall accuracy & performance** | ✅ **XGBoost (88%)** | Highest accuracy, best balance of precision and recall. |
| **Best recall for churn (finding most churners)** | ✅ **Gradient Boosting (0.60 recall)** | Finds more actual churn cases than XGBoost. |
| **Best precision for churn (reducing false churn predictions)** | ✅ **XGBoost & Gradient Boosting (0.44)** | Fewer false positives. |
| **Fastest training time & interpretation** | ✅ **Random Forest** | Easier to train and interpret. |
| **Best ROC-AUC Score (ranking ability)** | ✅ **Gradient Boosting (0.75)** | Best at ranking churn cases correctly. |

---

### **🚀 Final Recommendation: Choose Between Gradient Boosting & XGBoost**
🔹 **If you want the best recall for churn prediction → Go with Gradient Boosting.**  
🔹 **If you want the highest accuracy and better overall performance → Choose XGBoost.**  



### **Final Model Selection: Gradient Boosting 🚀**

Gradient Boosting has been chosen as the final model because it **prioritizes recall**, meaning it aims to identify as many potential churners as possible. While this approach may result in some false positives—predicting that a customer will churn when they actually won’t—it ensures that fewer actual churners are missed.

If the primary goal is to **minimize false negatives** (i.e., correctly identifying customers who are likely to leave), Gradient Boosting is the better choice. This makes it particularly useful for businesses that want to take proactive measures to retain customers before they churn.

---

### **Saving the Model to a Pickle File (`.pkl`)**
Here’s the code to **save the trained Gradient Boosting model** into a `.pkl` file so you can use it later:


In [ ]:
import pickle

# Save the trained Gradient Boosting model
with open("gradient_boosting_model.pkl", "wb") as file:
    pickle.dump(gradientboost, file)

print("Gradient Boosting model saved successfully!")

Gradient Boosting model saved successfully!
